In [6]:
import tensorflow as tf
import pickle
from sklearn.preprocessing import OneHotEncoder, LabelEncoder , StandardScaler
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [7]:
df = pd.read_csv('data\Churn_Modelling.csv')
df.head(2)

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Preeti Kumari\AppData\Local\Temp\ipykernel_21040\2113903702.py:1: SyntaxWarning: invalid escape sequence '\C'
  df = pd.read_csv('data\Churn_Modelling.csv')


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [8]:
cols = ['RowNumber', 'CustomerId','Surname']
df.drop(columns = cols , inplace = True , axis = 1)

In [9]:
df.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [10]:
label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])

In [11]:
onehot_encoder_geo = OneHotEncoder(handle_unknown = 'ignore')
geo_encoded = onehot_encoder_geo.fit_transform(df[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns = onehot_encoder_geo.get_feature_names_out(['Geography']))



In [12]:
df = pd.concat([df.drop('Geography', axis =1 ) , geo_encoded_df], axis = 1)

In [13]:
df.head(2)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0


In [14]:
#Dataset Splitting
X = df.drop('EstimatedSalary', axis =1)
y = df['EstimatedSalary']


In [15]:
X_train ,X_test, y_train , y_test = train_test_split(X, y , test_size = 0.2 , random_state = 42)

In [16]:
# with open('label_encoder_gender.pkl', 'wb') as file:
#     pickle.dump(label_encoder_gender, file)

# with open('ohe_geo.pkl', 'wb') as file:
#     pickle.dump(ohe_geo, file)

# ANN Regression

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [20]:
model = Sequential([
    Dense(64 , activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32 , activation='relu'),
    Dense(1) #Automatically linear activation fun is being applied for regression
])


# compiling the model
model.compile(optimizer = 'adam', loss = 'mean_absolute_error', metrics = ['mae'])

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard
import datetime


log_dir = 'regression/fit' + datetime.datetime.now().strftime('%Y%m$d_%H%M%S')
tensorboard_callback = TensorBoard(log_dir = log_dir , histogram_freq=1)

In [25]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights=True)

In [27]:
history = model.fit(
    X_train , y_train,
    epochs = 100,
    validation_data = (X_test, y_test),
    callbacks = [early_stopping_callback, tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 76294.0859 - mae: 76294.0859 - val_loss: 65500.1562 - val_mae: 65500.1562
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 64125.5898 - mae: 64125.5898 - val_loss: 55562.4023 - val_mae: 55562.4023
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 54413.6953 - mae: 54413.6953 - val_loss: 51187.8594 - val_mae: 51187.8594
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 51652.6758 - mae: 51652.6758 - val_loss: 52559.3711 - val_mae: 52559.3711
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 52143.3789 - mae: 52143.3789 - val_loss: 50555.8906 - val_mae: 50555.8906
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 51470.0703 - mae: 51470.0703 - val_loss: 50980.5195 - val_mae: 50980.5195
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 51250.6797 - mae: 51250.6797 - val_loss: 51052.1914 - val_mae: 51052.1914
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - 

In [29]:
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'The test mae is : {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 862us/step - loss: 50839.0195 - mae: 50839.0195
The test mae is : 50306.22265625


In [30]:
model.save('regression_model.h5')